type  model  
acad  Meta       22.290507
      chunk_1    23.002561
      chunk_2    22.969536
      gpt        24.708105
fic   Meta       25.658244
      chunk_1    24.808302
      chunk_2    24.817924
      gpt        26.350559
Name: Amount, dtype: float64


In [1]:
import os
import pandas as pd

# Load the CSV file with the correct encoding and specify headers
file_path = 'texts_2024_1120_aggregate.csv'

# Define the headers
headers = ['fullname', 'Measures', 'Amount', 'Level', 'Function'] 

df = pd.read_csv(file_path, encoding='latin1', names=headers, header=0)
df['filename'] = df['fullname'].str.split('/').str[-1]

# Extract type, id, and model
df[['type', 'id', 'model']] = df['filename'].str.extract(r'(\w+)_(\d+)@(\w+)')

In [2]:
df.head()

,fullname,Measures,Amount,Level,Function,filename,type,id,model
0,texts_2024_1120_results/chunk_1_acad/acad_0007...,Letter count (Par Avg),2731.0000,paragraph,average,acad_0007@chunk_1.csv,acad,0007,chunk_1
1,texts_2024_1120_results/chunk_1_acad/acad_0007...,Letter count (Par SD),NaN,paragraph,stdev,acad_0007@chunk_1.csv,acad,0007,chunk_1
2,texts_2024_1120_results/chunk_1_acad/acad_0007...,Letter count (Par Max),2731.0000,paragraph,maximum,acad_0007@chunk_1.csv,acad,0007,chunk_1
3,texts_2024_1120_results/chunk_1_acad/acad_0007...,Letter count (Par Min),2731.0000,paragraph,minimum,acad_0007@chunk_1.csv,acad,0007,chunk_1
4,texts_2024_1120_results/chunk_1_acad/acad_0007...,Letter count (Sent Avg),170.6875,sentence,average,acad_0007@chunk_1.csv,acad,0007,chunk_1


In [20]:
#measures_to_compare = unique[1::5][:50]
filtered_measures = [measure for measure in df['Measures'] if '(Par ' not in measure]
print(filtered_measures[:15])
# Prepare DataFrames for comparison
df = df[df['Measures'].isin(filtered_measures)]


['Letter count (Sent Avg)', 'Letter count (Sent SD)', 'Letter count (Sent Max)', 'Letter count (Sent Min)', 'Word count (Doc)', 'Word count (Sent Avg)', 'Word count (Sent SD)', 'Word count (Sent Max)', 'Word count (Sent Min)', 'Type count (Doc)', 'Type count (Sent Avg)', 'Type count (Sent SD)', 'Type count (Sent Max)', 'Type count (Sent Min)', 'Sentence count (Doc)']


In [21]:
grouped = df.groupby(['type', 'model', 'Measures'])

# Calculate the mean for each feature in the group
mean_features = grouped['Amount'].mean()

# Print the results
print(mean_features)


type  model  Measures                         
acad  Meta   Adjective avg position (Doc)         0.490135
             Adjective avg position (Sent Avg)    0.392278
             Adjective avg position (Sent Max)    0.819249
             Adjective avg position (Sent Min)    0.048171
             Adjective avg position (Sent SD)     0.254250
                                                    ...   
fic   gpt    Zipf frequency (Sent Max)            6.061212
             Zipf frequency (Sent Min)            4.094123
             Zipf frequency (Sent SD)             0.444964
             Zipf goodness of fit (Doc)           0.945431
             Zipf steepness of curve (Doc)        0.677113
Name: Amount, Length: 14224, dtype: float64


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming df is your DataFrame
# Filter data for chunk_1 and other models
chunk_1_data = df[mean_features['model'] == 'chunk_1']
other_models_data = df[df['model'] != 'chunk_1']

# Merge the data on 'type' and 'Measures'
merged_data = pd.merge(chunk_1_data, other_models_data, on=['type', 'Measures'], suffixes=('_chunk_1', '_other'))

# Calculate the difference
merged_data['Difference'] = merged_data['Amount_chunk_1'] - merged_data['Amount_other']

# Plot the differences
plt.figure(figsize=(14, 8))
sns.barplot(data=merged_data, x='Measures', y='Difference', hue='type')
plt.xticks(rotation=90)
plt.title('Differences in Measures between chunk_1 and Other Models')
plt.xlabel('Measures')
plt.ylabel('Difference')
plt.legend(title='Type')
plt.tight_layout()
plt.show()

TypeError: 'DataFrameGroupBy' object does not support item assignment

In [28]:
# Get unique values for each group
unique_values = grouped.apply(lambda x: x['model'].unique())

# Print the results
print(unique_values)

type  model  Measures                         
acad  Meta   Adjective avg position (Doc)         [Meta]
             Adjective avg position (Sent Avg)    [Meta]
             Adjective avg position (Sent Max)    [Meta]
             Adjective avg position (Sent Min)    [Meta]
             Adjective avg position (Sent SD)     [Meta]
                                                   ...  
fic   gpt    Zipf frequency (Sent Max)             [gpt]
             Zipf frequency (Sent Min)             [gpt]
             Zipf frequency (Sent SD)              [gpt]
             Zipf goodness of fit (Doc)            [gpt]
             Zipf steepness of curve (Doc)         [gpt]
Length: 14224, dtype: object


C:\Users\20191882\AppData\Local\Temp\ipykernel_20248\3540131418.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  unique_values = grouped.apply(lambda x: x['model'].unique())


In [ ]:
mean_features['model']=='chunk_1'